In [1]:
import pandas as pd
import sql_functions as sql_int

In [2]:
tournament_result = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/results_GSC1.csv')

In [3]:
tournament_result

,playerName,points,round,group,tournament
0,UTO Qetzer,7,1,A,Rising Legends Runeterra Reforged - Golden Spa...
1,Finntasy,2,1,A,Rising Legends Runeterra Reforged - Golden Spa...
2,Salvyyy,5,1,A,Rising Legends Runeterra Reforged - Golden Spa...
3,MK 5454,3,1,A,Rising Legends Runeterra Reforged - Golden Spa...
4,Talelelelelelel,8,1,A,Rising Legends Runeterra Reforged - Golden Spa...
...,...,...,...,...,...
1339,MIH ZyK0o,5,6,D,Rising Legends Runeterra Reforged - Golden Spa...
1340,Birderp,4,6,D,Rising Legends Runeterra Reforged - Golden Spa...
1341,l3s coco,6,6,D,Rising Legends Runeterra Reforged - Golden Spa...
1342,hyakuya,3,6,D,Rising Legends Runeterra Reforged - Golden Spa...


#### changing points into placement

In [4]:
df_tournament_result = tournament_result.copy()
df_tournament_result = df_tournament_result.dropna()
if 'placement' in df_tournament_result.columns: 
    pass 
else: 
    df_tournament_result['placement'] = df_tournament_result['points'].apply(lambda x: 9-int(x))
    df_tournament_result = df_tournament_result.drop(columns='points')
           
df_tournament_result[df_tournament_result.duplicated(keep= False, subset=['tournament','group','round','placement']) == True].sort_values(by=['tournament','round','group'])

,playerName,round,group,tournament,placement


In [5]:
df_tournament_result = df_tournament_result.drop_duplicates()

#### create Result tables

In [6]:
df_tournament_result = df_tournament_result.drop_duplicates()

df_lobby_result = (pd.pivot_table(df_tournament_result,values='playerName',columns='placement',index=['tournament','round','group'],aggfunc='first')
                   .reset_index()
                   .rename(columns={1:'winner',2:'runnerup',3:'third',4:'fourth',5:'fifth',6:'sixth',7:'seventh',8:'eighth'})
                   )
df_lobby_result

placement,tournament,round,group,winner,runnerup,third,fourth,fifth,sixth,seventh,eighth
0,Rising Legends Runeterra Reforged - Golden Spa...,1,A,Talelelelelelel,UTO Qetzer,TSAROU WINS,Salvyyy,omino blue,MK 5454,Finntasy,ancalimon
1,Rising Legends Runeterra Reforged - Golden Spa...,1,B,Kingdanzz,Cynarr,Twitch MemooTFT,Nevada34,Shinshire,stopteqh,TheAware,Eboy Urg0d
2,Rising Legends Runeterra Reforged - Golden Spa...,1,C,arzootft,Sologesang,Catalyst TFT,noeltft,AEG PasDeBol,nrvn5,LiquidSnoodyboo,burritto
3,Rising Legends Runeterra Reforged - Golden Spa...,1,D,DarkHydra,S1C xus0o,Lezzaford,HASHIDA,Portuguese Babe,Staig Rovskagg,Spoti,Littl3BitPr0
4,Rising Legends Runeterra Reforged - Golden Spa...,1,E,Wet Jungler,Aexoq,KURFUZZLED,hyakuya,Shener Shen,PMK Waegga,DARKlegendd54,immoq
...,...,...,...,...,...,...,...,...,...,...,...
163,Rising Legends Runeterra Reforged - Golden Spa...,5,D,TheAware,Helsingg,l3s coco,Cynarr,MIH ZyK0o,TFT Hyrule,Jerke,hyakuya
164,Rising Legends Runeterra Reforged - Golden Spa...,6,A,Skipaeus,Staig Rovskagg,MK 5454,Enzosx,kubixon,Opale,MattbLack,ttv SnubleTFT
165,Rising Legends Runeterra Reforged - Golden Spa...,6,B,Ging GG1,Sologesang,DarkHydra,nt mb,IZI Lyyyress,TG Fifek,Tomino6,z10 lelouch
166,Rising Legends Runeterra Reforged - Golden Spa...,6,C,Cynarr,OPS Jimray3,Littl3BitPr0,Helsingg,TheAware,Gunmay,Catalyst TFT,Diaja


#### Take tournament ID's

In [7]:
schema = 'tft_tables'
sql_query = f'''select name, tournament_id
                FROM tournaments;
                '''

df_tounament_id = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                    columns=['tournament','tournament_id'])

df_tounament_push = (pd.merge(df_lobby_result,df_tounament_id, how='left',left_on='tournament',right_on='tournament')
                     .drop(columns='tournament')
                     )

#### Checking tournament result already exists

In [8]:
sql_query = f'''SELECT tournament_id,round,party
                FROM lobbys;
                '''
df_existing_lobby = pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                    columns=['tournament_id','round','group'])
df_existing_lobby['existing'] = 'Yes'
df_no_dups = pd.concat([df_tounament_push,df_existing_lobby],ignore_index=True)
df_no_dups = df_no_dups.drop_duplicates(subset=['tournament_id','group','round'],keep=False).rename(columns={'group': 'party'})

df_no_dups = (df_no_dups
                  .drop_duplicates(subset=['tournament_id','round','party'],keep=False)
                  .query('existing != "Yes"')
                  .drop(['existing'],axis=1)
                  )

In [9]:
df_no_dups

,round,party,winner,runnerup,third,fourth,fifth,sixth,seventh,eighth,tournament_id
0,1,A,Talelelelelelel,UTO Qetzer,TSAROU WINS,Salvyyy,omino blue,MK 5454,Finntasy,ancalimon,6
1,1,B,Kingdanzz,Cynarr,Twitch MemooTFT,Nevada34,Shinshire,stopteqh,TheAware,Eboy Urg0d,6
2,1,C,arzootft,Sologesang,Catalyst TFT,noeltft,AEG PasDeBol,nrvn5,LiquidSnoodyboo,burritto,6
3,1,D,DarkHydra,S1C xus0o,Lezzaford,HASHIDA,Portuguese Babe,Staig Rovskagg,Spoti,Littl3BitPr0,6
4,1,E,Wet Jungler,Aexoq,KURFUZZLED,hyakuya,Shener Shen,PMK Waegga,DARKlegendd54,immoq,6
...,...,...,...,...,...,...,...,...,...,...,...
163,5,D,TheAware,Helsingg,l3s coco,Cynarr,MIH ZyK0o,TFT Hyrule,Jerke,hyakuya,8
164,6,A,Skipaeus,Staig Rovskagg,MK 5454,Enzosx,kubixon,Opale,MattbLack,ttv SnubleTFT,8
165,6,B,Ging GG1,Sologesang,DarkHydra,nt mb,IZI Lyyyress,TG Fifek,Tomino6,z10 lelouch,8
166,6,C,Cynarr,OPS Jimray3,Littl3BitPr0,Helsingg,TheAware,Gunmay,Catalyst TFT,Diaja,8


In [10]:
sql_int.push_to_database(df_no_dups,'lobbys',sql_int.get_engine_alchemy(),'tft_tables')

(mysql.connector.errors.IntegrityError) 1452 (23000): Cannot add or update a child row: a foreign key constraint fails (`tft_tables`.`lobbys`, CONSTRAINT `FK_SeventhName` FOREIGN KEY (`seventh`) REFERENCES `alternative_names` (`player_name`))
[SQL: INSERT INTO tft_tables.lobbys (round, party, winner, runnerup, third, fourth, fifth, sixth, seventh, eighth, tournament_id) VALUES (%(round_m0)s, %(party_m0)s, %(winner_m0)s, %(runnerup_m0)s, %(third_m0)s, %(fourth_m0)s, %(fifth_m0)s, %(sixth_m0)s, %(seventh_m0)s, %(eighth_m0)s, %(tournament_id_m0)s), (%(round_m1)s, %(party_m1)s, %(winner_m1)s, %(runnerup_m1)s, %(third_m1)s, %(fourth_m1)s, %(fifth_m1)s, %(sixth_m1)s, %(seventh_m1)s, %(eighth_m1)s, %(tournament_id_m1)s), (%(round_m2)s, %(party_m2)s, %(winner_m2)s, %(runnerup_m2)s, %(third_m2)s, %(fourth_m2)s, %(fifth_m2)s, %(sixth_m2)s, %(seventh_m2)s, %(eighth_m2)s, %(tournament_id_m2)s), (%(round_m3)s, %(party_m3)s, %(winner_m3)s, %(runnerup_m3)s, %(third_m3)s, %(fourth_m3)s, %(fifth_m3)s, 

In [15]:
df_control = df_no_dups['seventh'].apply(lambda x : x.lower())

In [47]:
sql_query = f'''select player_name
                FROM alternative_names;
                '''

df_all_names= pd.DataFrame(sql_int.get_data_mysql(sql_query),
                                    columns=['alternative_names'])

In [48]:
df_all_names['alternative_names'] = df_all_names['alternative_names'].astype(str).apply(lambda x : x.lower())

In [55]:
df_all_names['sql'] = 'was there'

In [64]:
df_all_names[df_all_names['alternative_names'] == 'aug1']

,alternative_names,sql


In [58]:
df_control

,seventh
0,finntasy
1,theaware
2,liquidsnoodyboo
3,spoti
4,darklegendd54
...,...
138,mk 5454
139,ano shreddin
146,jerke
152,catalyst tft


In [59]:
df_hmmmm = pd.merge(df_control,df_all_names,how='left',left_on='seventh',right_on='alternative_names')

In [61]:
df_hmmmm[df_hmmmm['sql'] != 'was there']

,seventh,alternative_names,sql
11,aug1,NaN,NaN
68,mattblack,NaN,NaN
